# Capstone Project 01 - Segmentation and Clustering

In [90]:
#!conda install -y -c conda-forge geopy 

In [91]:
#!conda install -y -c conda-forge folium=0.5

In [92]:
#!conda install -y -c anaconda beautifulsoup4

In [11]:
#!conda install -y -c anaconda lxml

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Marcio\Miniconda3\envs\intel

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxslt-1.1.32             |       hf6f1972_0         458 KB  anaconda
    lxml-4.2.5                 |   py37hef2cd61_0         1.2 MB  anaconda
    ------------------------------------------------------------
                                           Total:         1.7 MB

The following NEW packages will be INSTALLED:

  libxslt            anaconda/win-64::libxslt-1.1.32-hf6f1972_0
  lxml               anaconda/win-64::lxml-4.2.5-py37hef2cd61_0




lxml-4.2.5   

In [14]:
!pip install lxml

# Importing Libs

In [1]:
import numpy as np 
import pandas as pd 

from bs4 import BeautifulSoup
from urllib.request import urlopen

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries importing, done!')

Libraries importing, done!


# 1. Getting Canada data from Wikipedia

Geting the data from the wikipedia site [https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [2]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
#soup = BeautifulSoup(html, features='lxml')
soup = BeautifulSoup(html, features='html.parser')

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})
data = []

for record in My_table.findAll('td'):
    data.append(record.text)
data[0:5]

['M1A', 'Not assigned', 'Not assigned\n', 'M2A', 'Not assigned']

### Generating a Dataframe 

In [4]:
PostalCode = data[0::3]
Borough = data[1::3]
Neighborhood = data[2::3]
Neighborhood = [line.rstrip('\n') for line in Neighborhood]

df = pd.DataFrame()
df['PostalCode'] = PostalCode
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood

df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [6]:
# combine row values of Neighborhood into one row that share the same postal code 
df = df.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg(','.join)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [7]:
# for Borough "Queen's Park", replace its Neighborhood column value with "Queen's Park"
df.replace('Not assigned', 'Queen\'s Park', inplace = True)

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [8]:
df.shape

(103, 3)